<a href="https://colab.research.google.com/github/brandontanzhirong/Learning-based-Multi-view-3D-Model-Reconstruction/blob/master/notebook/Backend_Util.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Endpoint URL

In [ ]:
!pip install colabcode
!pip install fastapi
# !pip install -U -q PyDrive
!pip install python-multipart

     |████████████████████████████████| 745 kB 3.9 MB/s 
     |████████████████████████████████| 8.3 MB 42.7 MB/s 
     |████████████████████████████████| 45 kB 2.8 MB/s 
     |████████████████████████████████| 396 kB 56.6 MB/s 
     |████████████████████████████████| 428 kB 55.5 MB/s 
     |████████████████████████████████| 60 kB 7.1 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 78 kB 5.4 MB/s 
     |████████████████████████████████| 129 kB 30.6 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 125 kB 51.0 MB/s 
     |████████████████████████████████| 124 kB 51.5 MB/s 
     |████████████████████████████████| 124 kB 48.0 MB/s 
     |████████████████████████████████| 122 kB 47.3 MB/s 
     |████████████████████████████████| 122 kB 56.6 MB/s 
     |████████████████████████████████| 122 kB 56.7 MB/s 
     |████████████████████████████████| 122 kB 56.5 MB/s 
     |███████████████████

     |████████████████████████████████| 51 kB 531 kB/s 
     |████████████████████████████████| 10.9 MB 6.7 MB/s 
     |████████████████████████████████| 61 kB 232 kB/s 
  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31678 sha256=9e67c5d8a21a828ddb1eb8114457c455028229484d72c609f0979fbeacdb7f21
  Stored in directory: /root/.cache/pip/wheels/2c/41/7c/bfd1c180534ffdcc0972f78c5758f89881602175d48a8bcd2c
Successfully built python-multipart


In [ ]:
from typing import List
from colabcode import ColabCode
from fastapi import FastAPI, File, UploadFile, Response, Form
import pickle
from pydantic import BaseModel
import os
from fastapi.middleware.cors import CORSMiddleware
from PIL import Image
import io
from fastapi.staticfiles import StaticFiles

MUSEUM_COLLECTIONS_FOLDER_ID = "10O-Ar1Kwqru6KPTWktyLZ7399R_6s4QZ"

class NewFolder(BaseModel):
  model_name: str

app = FastAPI()
origins = ["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

app.mount("/static", StaticFiles(directory="/content/drive/MyDrive/FYP/Sample 3D Model/Museum Collections"), name="static")

@app.post('/new-folder')
def create_new_model_folder(new_folder: NewFolder):
  new_folder_dict = new_folder.dict()
  model_name = new_folder_dict["model_name"]
  model_folder = drive.ListFile({'q': f"'{MUSEUM_COLLECTIONS_FOLDER_ID}' in parents and trashed=false and title='{model_name}'"}).GetList()
  if len(model_folder) > 0:
    model_folder_id = model_folder[0]["id"]
    file_list = drive.ListFile({'q': f"'{model_folder_id}' in parents and trashed=false"}).GetList()
    for file1 in file_list:
      if file1["title"] == "images":
        img_folder_id = file1["id"]
        break
    return {'status_code': 400, "folder_id": img_folder_id}
  model_folder = drive.CreateFile({'title' : model_name, 'mimeType' : 'application/vnd.google-apps.folder', 'parents': [{'id': MUSEUM_COLLECTIONS_FOLDER_ID}]})
  model_folder.Upload()
  img_folder = drive.CreateFile({'title' : "images", 'mimeType' : 'application/vnd.google-apps.folder', 'parents': [{'id': model_folder["id"]}]})
  img_folder.Upload()
  return {'status_code': 201, "folder_id": img_folder["id"]}

@app.post('/upload-images')
def create_upload_files(img_files: List[UploadFile] = File(...), img_folder_id: str = Form(...)):
  os.makedirs("temp", exist_ok = True)
  for img_file in img_files:
    file_drive = drive.CreateFile({'title':img_file.filename, 'mimeType': img_file.content_type, 'parents': [{'id': img_folder_id}]})
    with open("temp/"+img_file.filename, 'wb') as image:
      content = img_file.file.read()
      image.write(content)
      image.close()
    file_drive.SetContentFile("temp/"+img_file.filename)
    file_drive.Upload()
  return {"status_code": 201}

@app.get('/img-folder')
def get_image_file_ids(model_name : str):
  model_folder = drive.ListFile({'q': f"'{MUSEUM_COLLECTIONS_FOLDER_ID}' in parents and trashed=false and title='{model_name}'"}).GetList()[0]
  model_folder_id = model_folder["id"]
  file_list = drive.ListFile({'q': f"'{model_folder_id}' in parents and trashed=false"}).GetList()
  for file1 in file_list:
    if file1["title"] == "images":
      img_folder_id = file1["id"]
      break
  file_list = drive.ListFile({'q': f"'{img_folder_id}' in parents and trashed=false"}).GetList()
  img_details = [{"title": file1["title"], "thumbnailLink": file1["thumbnailLink"]} for file1 in file_list]
  return img_details

@app.get('/get-model-list')
def get_model_list():
  model_list = drive.ListFile({'q': f"'{MUSEUM_COLLECTIONS_FOLDER_ID}' in parents and trashed=false"}).GetList()
  models = [model["title"] for model in model_list]
  return models

@app.get('/check-model-progress')
def check_model_progress():
  model_list = drive.ListFile({'q': f"'{MUSEUM_COLLECTIONS_FOLDER_ID}' in parents and trashed=false"}).GetList()
  models = [model["title"] for model in model_list]
  return models


server = ColabCode(port=10000, code=False, mount_drive=True)
server.run_app(app=app)

Public URL: NgrokTunnel: "https://add9-35-221-174-180.ngrok.io" -> "http://localhost:10000"


INFO:     Started server process [741]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:10000 (Press CTRL+C to quit)


INFO:     2001:e68:541b:e500:c1c5:fa1c:efe5:a2ba:0 - "GET /static/Ganesha/result_textured/result_textured.mtl HTTP/1.1" 200 OK
INFO:     2001:e68:541b:e500:c1c5:fa1c:efe5:a2ba:0 - "GET /static/Ganesha/result_textured/result_textured.obj HTTP/1.1" 200 OK
INFO:     2001:e68:541b:e500:c1c5:fa1c:efe5:a2ba:0 - "GET /static/Ganesha/result_textured/result_textured_material0002_map_Kd.png HTTP/1.1" 200 OK
INFO:     2001:e68:541b:e500:c1c5:fa1c:efe5:a2ba:0 - "GET /static/Ganesha/result_textured/result_textured_material0001_map_Kd.png HTTP/1.1" 200 OK
INFO:     2001:e68:541b:e500:c1c5:fa1c:efe5:a2ba:0 - "GET /static/Ganesha/result_textured/result_textured_material0000_map_Kd.png HTTP/1.1" 200 OK
INFO:     2001:e68:541b:e500:c1c5:fa1c:efe5:a2ba:0 - "GET /get-model-list HTTP/1.1" 200 OK
INFO:     2001:e68:541b:e500:c1c5:fa1c:efe5:a2ba:0 - "GET /static/Ganesha/result_textured/result_textured_material0003_map_Kd.png HTTP/1.1" 200 OK
INFO:     2001:e68:541b:e500:c1c5:fa1c:efe5:a2ba:0 - "GET /static/G

In [ ]:
  model_folder = drive.CreateFile({'title' : "test", 'mimeType' : 'application/vnd.google-apps.folder', 'parents': [{'id': MUSEUM_COLLECTIONS_FOLDER_ID}]})
  model_folder.Upload()
  img_folder = drive.CreateFile({'title' : "images", 'mimeType' : 'application/vnd.google-apps.folder', 'parents': [{'id': model_folder["id"]}]})
  img_folder.Upload()
  img_folder["id"]

In [ ]:
def get_image_file_ids(model_name : str):
  MUSEUM_COLLECTIONS_FOLDER_ID = "10O-Ar1Kwqru6KPTWktyLZ7399R_6s4QZ"
  model_folder = drive.ListFile({'q': f"'{MUSEUM_COLLECTIONS_FOLDER_ID}' in parents and trashed=false and title='{model_name}'"}).GetList()[0]
  model_folder_id = model_folder["id"]
  file_list = drive.ListFile({'q': f"'{model_folder_id}' in parents and trashed=false"}).GetList()
  for file1 in file_list:
    if file1["title"] == "images":
      img_folder_id = file1["id"]
      break
  file_list = drive.ListFile({'q': f"'{img_folder_id}' in parents and trashed=false"}).GetList()
  img_ids = [file1 for file1 in file_list]
  return img_ids[0]

In [ ]:
file1 = get_image_file_ids(model_name="Vase")

In [ ]:
import pprint
pprint.pprint(file1)